In [1]:
from __future__ import print_function, division

In [2]:
import pandas as pd
df = pd.read_csv('data.csv', sep=';')
df.head()

,TV,Description
0,M,kohout kulový plnoprůtokový nikl ovládání páč...
1,M,8/8 DÍRY KOLENO PATNÍ PŘÍRUBOVÉ 80 - 8/8 DÍRY
2,M,"ABSOLUT - souprava distančních prvků, světlý p..."
3,M,"ABSOLUT - souprava distančních prvků, světlý p..."
4,M,"ABSOLUT - souprava napojení horních dvířek, sv..."


In [3]:
data = df['Description']
target = df['TV'].replace('K', 0).replace('M', 1)
names = ['K', 'M']
print(data[:5])
print(target[:5])

0     kohout kulový plnoprůtokový nikl ovládání páč...
1        8/8 DÍRY KOLENO PATNÍ PŘÍRUBOVÉ 80 - 8/8 DÍRY
2    ABSOLUT - souprava distančních prvků, světlý p...
3    ABSOLUT - souprava distančních prvků, světlý p...
4    ABSOLUT - souprava napojení horních dvířek, sv...
Name: Description, dtype: object
0    1
1    1
2    1
3    1
4    1
Name: TV, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)
print('Train size: {}'.format(len(X_train)))
print('Test size: {}'.format(len(X_test)))

Train size: 104924
Test size: 26232


In [5]:
from nltk.tokenize.casual import casual_tokenize

item = data[4]
print(item)

tokenizer = lambda text: casual_tokenize(text, preserve_case=False)

print(tokenizer(item))

ABSOLUT - souprava napojení horních dvířek, světlý průřez 12 cm
['absolut', '-', 'souprava', 'napojení', 'horních', 'dvířek', ',', 'světlý', 'průřez', '12', 'cm']


In [6]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

stem_tokenizer = lambda text: [stemmer.stem(w) for w in tokenizer(text)]

print (stem_tokenizer(item))

['absolut', '-', 'souprava', 'napojení', 'horních', 'dvířek', ',', 'světlý', 'průřez', '12', 'cm']


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=stem_tokenizer)
vectorizer.fit(X_train)
print (vectorizer.transform([item]))

  (0, 9)	1
  (0, 10)	1
  (0, 1544)	1
  (0, 10408)	1
  (0, 11758)	1
  (0, 12957)	1
  (0, 14222)	1
  (0, 18270)	1
  (0, 22691)	1
  (0, 25634)	1
  (0, 26705)	1


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(vectorizer.transform(X_train))
print(tfidf_transformer.transform(vectorizer.transform([item])))

  (0, 26705)	0.2744615727763041
  (0, 25634)	0.3534263029909493
  (0, 22691)	0.29336091621481336
  (0, 18270)	0.3393481134279839
  (0, 14222)	0.37959510047242817
  (0, 12957)	0.4168622964955362
  (0, 11758)	0.1832268406565983
  (0, 10408)	0.37827948714349513
  (0, 1544)	0.25232910591279983
  (0, 10)	0.17982519480474665
  (0, 9)	0.08945270104202255


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier

clf_pipeline = Pipeline([('vec', vectorizer),
                         ('tfidf', tfidf_transformer),
                         #('lr', LogisticRegression())
                         #('gbc', GradientBoostingClassifier(n_estimators=100, max_depth=4))
                         ('svm', svm.SVC(kernel='linear'))
                        ])
clf_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [10]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

y_pred = clf_pipeline.predict(X_test)

print ("Test accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))
print ()
print(metrics.classification_report(y_test, y_pred, digits=4))

Test accuracy: 1.00

             precision    recall  f1-score   support

          0     0.9980    0.9966    0.9973     17607
          1     0.9932    0.9959    0.9946      8625

avg / total     0.9964    0.9964    0.9964     26232



In [11]:
y_pred = clf_pipeline.predict(X_train)

print ("Train accuracy: {:.2f}".format(accuracy_score(y_train, y_pred)))
print ()
print(metrics.classification_report(y_train, y_pred, digits=4))
print(y_train[:50])
print(y_pred[:50])

Train accuracy: 1.00

             precision    recall  f1-score   support

          0     0.9997    0.9990    0.9994     70122
          1     0.9981    0.9993    0.9987     34802

avg / total     0.9991    0.9991    0.9991    104924

120350    0
78423     0
17080     1
40977     0
107245    1
91225     1
26298     1
50951     0
41715     0
12219     0
31143     0
116349    1
52455     0
16479     0
46661     0
40968     0
11975     1
47794     0
25171     1
25838     1
27892     1
30924     0
90965     1
114754    0
85882     0
125692    0
54528     0
12795     0
32602     0
44944     0
117588    1
40954     0
74592     1
85106     0
11023     1
2808      1
119495    0
103111    1
98329     1
9035      0
110583    1
5538      0
17151     0
42363     0
8315      0
109231    1
61231     1
31497     0
89344     1
101961    1
Name: TV, dtype: int64
[0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0
 1 1 0 1 0 0 0 0 1 1 0 1 1]


In [12]:
print(X_test[:5])
print(y_pred[:5])

10209                 deska izolační krbová 250x1000x100 mm
98105     sopouch 0° (se sponou) DN 150 mm - komínový tř...
67646     Opravy dopravníků a dopravních zařízení Montáž...
98330     souprava zemní pro šoupátka DN 100-150 mm, Rd ...
117169    Vodorovné přemístění větví, kmenů nebo pařezů ...
Name: Description, dtype: object
[0 0 1 0 1]


In [13]:
text = ["Žlaby šachet z cihel z kyselinovzdorné kameniny PKZ normálek N 65 průměru do 500 mm",
       "žlaby z taveného čediče čtvrtinové l 500mm tl 20mm D 303mm",
       "Obklady schodišť přírodním litým teracem  stupňů tl. do 25 mm se zábradlím profilovaných",
       "Oblouk 45° PE100 RC SDR17 110",
       "Oblouk 47° PE255 RC",
       "Oblozeni sten z cementotriskovych desek sroubovanych"]

res = list(map(lambda x: "K" if x == 0 else "M", clf_pipeline.predict(text)))

print(res)

['K', 'M', 'K', 'M', 'M', 'K']


In [14]:
testing = pd.read_csv('testing.csv', sep=';')
testingData = testing['Description']
testingResult = testing['TV']

testPredictPlain = clf_pipeline.predict(testingData)
testPrediction = list(map(lambda x: "K" if x == 0 else "M", testPredictPlain))

print(len(testingData))

for index, item in enumerate(testPrediction):
    print (item + "->" + testingResult[index] + 3*" " + testingData[index])

131
K->K   Asfaltový beton vrstva obrusná ACO 11 (ABS)  s rozprostřením a se zhutněním z modifikovaného asfaltu v pruhu šířky přes 3 m tl. 50 mm
K->K   Bednění mostních pilířů a sloupů konstantního průřezu ze systémového bednění  zřízení pro stativa a příčníky
K->K   Bourání plotových sloupků a vzpěr železobetonových výšky do 2,5 m s betonovou patkou
K->K   Čerpadla teplovodní závitová mokroběžná oběhová pro teplovodní vytápění (elektronicky řízená) PN 10, do 110°C DN přípojky/dopravní výška H (m) - čerpací výkon Q (m3/h) DN 25 / do 6,0 m / 7,0 m3/h
K->K   Demolice hal průmyslových, zemědělských nebo občanské výstavby  těžkými mechanizačními prostředky z cihel, tvárnic, kamene, zdiva smíšeného nebo hrázděného na maltu vápennou nebo vápenocementovou s podílem konstrukcí do 10 %
K->K   Demontáž lešení prostorového modulového těžkého pracovního nebo podpěrného bez podlah  s provozním zatížením tř. 6 přes 450 do 600 kg/m2, výšky do 10 m
K->K   Demontáž ocelového potrubí do šrotu hmotnosti 

In [15]:
testResultVec = list(map(lambda x: 0 if x == "K" else 1, testingResult))
print ("Train accuracy: {:.2f}".format(accuracy_score(testResultVec, testPredictPlain)))

Train accuracy: 0.98
